In [ ]:
import pandas as pd
import numpy as np

In [ ]:
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

unit_size = 256
layers = "GRU"


## Results from the paper

In [ ]:
GRU = {
    'mort_hosp':{'auc': 0.8504, 'auprc': 0.5215, 'F1': 0.4229},
    'mort_icu': {'auc': 0.8632, 'auprc': 0.4651, 'F1': 0.3630},
    'los_3':    {'auc': 0.6740, 'auprc': 0.6017, 'F1': 0.5336},
    'los_7':    {'auc': 0.7054, 'auprc': 0.1625, 'F1': 0.0233},
}

# Averaged Multimodal

word2vec_avg = {
    'mort_hosp':{'auc': 0.8642, 'auprc': 0.5422, 'F1': 0.4542},
    'mort_icu': {'auc': 0.8717, 'auprc': 0.4847, 'F1': 0.4230},
    'los_3':    {'auc': 0.6863, 'auprc': 0.6181, 'F1': 0.5419},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1791, 'F1': 0.0135},
}

fastText_avg = {
    'mort_hosp':{'auc': 0.8609, 'auprc': 0.5447, 'F1': 0.4550},
    'mort_icu': {'auc': 0.8714, 'auprc': 0.4836, 'F1': 0.4291},
    'los_3':    {'auc': 0.6855, 'auprc': 0.6159, 'F1': 0.5446},
    'los_7':    {'auc': 0.7131, 'auprc': 0.1757, 'F1': 0.0102},
}

concat_avg = {
    'mort_hosp':{'auc': 0.8598, 'auprc': 0.5419, 'F1': 0.4566},
    'mort_icu': {'auc': 0.8690, 'auprc': 0.4828, 'F1': 0.4076},
    'los_3':    {'auc': 0.6861, 'auprc': 0.6169, 'F1': 0.5470},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1767, 'F1': 0.0137},
}


# Proposed Model

word2vec_proposed = {
    'mort_hosp':{'auc': 0.8755, 'auprc': 0.5587, 'F1': 0.4723},
    'mort_icu': {'auc': 0.8835, 'auprc': 0.4923, 'F1': 0.4302},
    'los_3':    {'auc': 0.6954, 'auprc': 0.6268, 'F1': 0.5504},
    'los_7':    {'auc': 0.7255, 'auprc': 0.1878, 'F1': 0.0158},
}

fastText_proposed = {
    'mort_hosp':{'auc': 0.8715, 'auprc': 0.5568, 'F1': 0.4687},
    'mort_icu': {'auc': 0.8785, 'auprc': 0.4878, 'F1': 0.4309},
    'los_3':    {'auc': 0.6961, 'auprc': 0.6255, 'F1': 0.5587},
    'los_7':    {'auc': 0.7181, 'auprc': 0.1801, 'F1': 0.0108},
}

concat_proposed = {
    'mort_hosp':{'auc': 0.8698, 'auprc': 0.5535, 'F1': 0.4638},
    'mort_icu': {'auc': 0.8766, 'auprc': 0.4874, 'F1': 0.4224},
    'los_3':    {'auc': 0.6993, 'auprc': 0.6277, 'F1': 0.5582},
    'los_7':    {'auc': 0.7192, 'auprc': 0.1825, 'F1': 0.0138},
}


In [ ]:
def print_results(model, target_problem, auc, auprc, F1):
    
    print("Target Problem: {}".format(target_problem))
    print("----------------------------")
    print("AUROC: {:.4f} {} {:.3f}   (Paper: {:.4f}, Abs Diff: {:.4f} ({:.1f} %))".format(
        np.mean(auc),   
        u"\u00B1", 
        np.std(auc),
        model[target_problem]['auc'],
        np.abs(np.round(np.mean(auc),4) - model[target_problem]['auc']),
        np.abs(np.round(np.mean(auc),4) - model[target_problem]['auc']) / model[target_problem]['auc'] * 100
    ))
    
    print("AUPRC: {:.4f} {} {:.3f}   (Paper: {:.4f}, Abs Diff: {:.4f} ({:.1f} %))".format(
        np.mean(auprc), 
        u"\u00B1", 
        np.std(auprc),
        model[target_problem]['auprc'],
        np.abs(np.round(np.mean(auprc),4) - model[target_problem]['auprc']),
        np.abs(np.round(np.mean(auprc),4) - model[target_problem]['auprc']) / model[target_problem]['auprc'] * 100
    ))
    
    print("F1:    {:.4f} {} {:.3f}   (Paper: {:.4f}, Abs Diff: {:.4f} ({:.1f} %))".format(
        np.mean(F1),    
        u"\u00B1", 
        np.std(F1),
        model[target_problem]['F1'],
        np.abs(np.round(np.mean(F1),4) - model[target_problem]['F1']),
        np.abs(np.round(np.mean(F1),4) - model[target_problem]['F1']) / model[target_problem]['F1'] * 100
    ))
    print("")

## GRU

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []
    
    for run in range(1,11):
        data = pd.read_pickle("results/07-GRU/{}-{}-{}-{}-new.p".format(
            unit_size, 
            layers, 
            target_problem, 
            run))

        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(GRU, target_problem, auc, auprc, F1)

# Averaged MultiModal models

## Word2Vec

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []
    
    for run in range(1,11):
        data = pd.read_pickle("results/08-multimodal/{}-{}-{}-{}-{}-new-avg-.p".format(
            layers, 
            unit_size, 
            'word2vec', 
            target_problem, 
            run))

        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(word2vec_avg, target_problem, auc, auprc, F1)

## FastText

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []
    
    for run in range(1,11):
        data = pd.read_pickle("results/08-multimodal/{}-{}-{}-{}-{}-new-avg-.p".format(
            layers, 
            unit_size, 
            'fasttext', 
            target_problem, 
            run))

        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(fastText_avg, target_problem, auc, auprc, F1)

## concat

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []
    
    for run in range(1,11):
        data = pd.read_pickle("results/08-multimodal/{}-{}-{}-{}-{}-new-avg-.p".format(
            layers, 
            unit_size, 
            'concat', 
            target_problem, 
            run))

        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(concat_avg, target_problem, auc, auprc, F1)

# Proposed Models

## word2vec

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []    
    
    for run in range(1,11):
        data = pd.read_pickle("results/09-cnn/{}-{}-{}-{}-{}-new-cnn-.p".format(
            layers, 
            unit_size, 
            'word2vec', 
            target_problem, 
            run))
        
        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(word2vec_proposed, target_problem, auc, auprc, F1)

## FastText

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []    
    
    for run in range(1,11):
        data = pd.read_pickle("results/09-cnn/{}-{}-{}-{}-{}-new-cnn-.p".format(
            layers, 
            unit_size, 
            'fasttext', 
            target_problem, 
            run))
        
        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(fastText_proposed, target_problem, auc, auprc, F1)

## concat

In [ ]:
for target_problem in target_problems:
    auc   = []
    auprc = []
    acc   = []
    F1    = []    
    
    for run in range(1,11):
        data = pd.read_pickle("results/09-cnn/{}-{}-{}-{}-{}-new-cnn-.p".format(
            layers, 
            unit_size, 
            'concat', 
            target_problem, 
            run))
        
        auc.append(data['auc'])
        auprc.append(data['auprc'])
        acc.append(data['acc'])
        F1.append(data['F1'])
        
    print_results(concat_proposed, target_problem, auc, auprc, F1)